In [1]:
import torch
from pathlib import Path
from PIL import Image, ImageDraw, ImageEnhance, ImageOps
import pytesseract
import os
import random
import torchvision.transforms as transforms
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def get_model_instance_segmentation(num_classes):
    weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=weights)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

model_path = "D:/test3/RCNN/best_model.pth"
if not os.path.exists(model_path):
    print(f"Model file not found at {model_path}")
else:
    print(f"Model file found at {model_path}")

# 加載已訓練好的Faster R-CNN模型
model = get_model_instance_segmentation(num_classes=2)
model.load_state_dict(torch.load(model_path))
model.eval()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# 設置測試集圖片路徑和結果保存路徑
test_image_folder = 'D:/test3/container_dataset/test/images'
results_folder = 'D:/test3/ocr_results'
recognized_images_folder = 'D:/test3/ocr_results/recognized_RCNN'
preprocessed_images_folder = 'D:/test3/ocr_results/preprocessed_RCNN'
os.makedirs(results_folder, exist_ok=True)
os.makedirs(recognized_images_folder, exist_ok=True)
os.makedirs(preprocessed_images_folder, exist_ok=True)

# 定義正確的貨櫃號碼，根據檔案名稱
correct_numbers_folder = 'D:/test3/picture_test_sataset/picture_test_sataset'
correct_numbers = {file_name: file_name.split('.')[0] for file_name in os.listdir(correct_numbers_folder)}

correct_count = 0
total_count = 0

def preprocess_image(img):
    # 增加對比度
    enhancer = ImageEnhance.Contrast(img)
    img = enhancer.enhance(2)
    
    # 轉換為灰度圖像
    img = ImageOps.grayscale(img)
    
    # 二值化處理
    img = img.point(lambda x: 0 if x < 128 else 255, '1')
    
    return img

preprocessed_images = []
recognized_image_paths = []

for image_file in os.listdir(test_image_folder):
    if image_file.endswith(".jpg") or image_file.endswith(".png"):
        image_path = os.path.join(test_image_folder, image_file)
        image = Image.open(image_path)
        
        # 使用Faster R-CNN模型進行物件偵測
        img = Image.open(image_path).convert("RGB")
        transform = transforms.Compose([transforms.ToTensor()])
        img = transform(img)
        img = img.to(device)
        
        with torch.no_grad():
            prediction = model([img])
        
        detected_objects = prediction[0]['boxes'].cpu().numpy()
        scores = prediction[0]['scores'].cpu().numpy()
        
        # 繪製檢測結果
        draw = ImageDraw.Draw(image)
        for i, box in enumerate(detected_objects):
            if scores[i] > 0.5:
                x1, y1, x2, y2 = box
                draw.rectangle([x1, y1, x2, y2], outline="red", width=2)
        
        # 保存視覺化結果
        image.save(os.path.join(results_folder, f"{image_file}_detected.jpg"))
        
        for i, box in enumerate(detected_objects):
            if scores[i] > 0.5:
                x1, y1, x2, y2 = box
                cropped_img = image.crop((x1, y1, x2, y2))
                
                # 預處理裁剪出的圖片
                preprocessed_img = preprocess_image(cropped_img)
                preprocessed_img_path = os.path.join(preprocessed_images_folder, f"{image_file}_preprocessed_{len(preprocessed_images)}.jpg")
                preprocessed_img.save(preprocessed_img_path)
                preprocessed_images.append(preprocessed_img_path)
                
                # 使用Tesseract進行文字辨識
                recognized_text = pytesseract.image_to_string(preprocessed_img)
                recognized_text = ''.join(filter(str.isalnum, recognized_text))
                
                # 比較前11位數字
                correct_number = correct_numbers.get(image_file, "")
                if recognized_text[:11] == correct_number[:11]:
                    correct_count += 1
                    recognized_image_path = os.path.join(recognized_images_folder, f"{image_file}_recognized.jpg")
                    image.save(recognized_image_path)
                    recognized_image_paths.append(recognized_image_path)
                total_count += 1

accuracy = correct_count / total_count if total_count > 0 else 0
print(f"OCR 準確率: {accuracy:.2f}")

print("成功辨識的圖片已保存至資料夾:")
for img_path in recognized_image_paths:
    print(img_path)

print("隨機顯示5張預處理過的圖片:")
for img_path in random.sample(preprocessed_images, min(5, len(preprocessed_images))):
    print(img_path)

Model file found at D:/test3/RCNN/best_model.pth
OCR 準確率: 0.82
成功辨識的圖片已保存至資料夾:
D:/test3/ocr_results/recognized_RCNN\image_0001.jpg_recognized.jpg
D:/test3/ocr_results/recognized_RCNN\image_0002.jpg_recognized.jpg
D:/test3/ocr_results/recognized_RCNN\image_0003.jpg_recognized.jpg
D:/test3/ocr_results/recognized_RCNN\image_0007.jpg_recognized.jpg
D:/test3/ocr_results/recognized_RCNN\image_0008.jpg_recognized.jpg
D:/test3/ocr_results/recognized_RCNN\image_0009.jpg_recognized.jpg
D:/test3/ocr_results/recognized_RCNN\image_0010.jpg_recognized.jpg
D:/test3/ocr_results/recognized_RCNN\image_0011.jpg_recognized.jpg
D:/test3/ocr_results/recognized_RCNN\image_0012.jpg_recognized.jpg
D:/test3/ocr_results/recognized_RCNN\image_0014.jpg_recognized.jpg
D:/test3/ocr_results/recognized_RCNN\image_0018.jpg_recognized.jpg
D:/test3/ocr_results/recognized_RCNN\image_0019.jpg_recognized.jpg
D:/test3/ocr_results/recognized_RCNN\image_0020.jpg_recognized.jpg
D:/test3/ocr_results/recognized_RCNN\image_0021.jp